# Web検索連動チャットボット（LangGraph版）の構築

## 1.準備

In [ ]:
from dotenv import load_dotenv
from typing import TypedDict, Annotated, Any
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.tools import tool
from langchain_tavily import TavilySearch
from IPython.display import Image, display

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
import uuid
from langgraph.checkpoint.memory import InMemorySaver
from langchain_core.messages import AIMessageChunk, BaseMessage

# 環境変数の読み込み
load_dotenv(".env")

# モデル名
MODEL_NAME = "gpt-5-mini"

## 2. ツールの定義

In [ ]:
# ツールの初期化
tavily_search = TavilySearch(
    max_results=2,                 # 取得する検索結果の数
    search_depth="basic",          # "basic" (高速) か "advanced" (高品質)
    include_answer=False,          # Tavilyが生成した短い回答を含めない
    include_raw_content=False,     # HTMLの生コンテンツを含めるか（トークン消費増に注意）
    include_images=False,          # 画像URLを含めるか
    # include_domains=["go.jp"],   # 特定のドメインのみ検索する場合
    # exclude_domains=["wikipedia.org"] # 特定のドメインを除外する場合
)

# Tavilyの検索結果だけを、LLMが読みやすい形に整形する関数
def format_tavily_results(tavily_response: dict) -> str:
    results = tavily_response.get("results", [])
    if not results:
        return "（検索結果なし）"

    lines = []
    for i, r in enumerate(results, 1):
        title = r.get("title", "")
        content = r.get("content", "")
        url = r.get("url", "")
        lines.append(f"[{i}] {title}\n{content}\nsource: {url}")
    return "\n\n".join(lines)

# TavilySearchの生レスポンス(dict/JSON)を整形して返すラッパーTool
@tool
def tavily_search_formatted(query: str) -> str:
    """Web検索（Tavily）。上位結果を整形して返す。"""
    tavily_response = tavily_search.invoke({"query": query})
    return format_tavily_results(tavily_response)

@tool
def internal_search(query: str) -> str:
    """（デモ用モック）Lucky社の社内規則を返す"""

    rules = """
        Lucky社 社内規則
        毎朝、鏡の中の自分に「今日はツイている」と宣言してから業務を始めること。
        同僚の成功を自分のことのように喜び、社内の「運気」を循環させること。
        どんなトラブルも、まずは「ラッキー！」と口に出してから解決策を考えること。
        """
    return rules

tools = [tavily_search_formatted, internal_search]

## 3. ノードとグラフの準備

In [ ]:
class State(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages]

# モデルの初期化
model = ChatOpenAI(model=MODEL_NAME)
model_with_tools = model.bind_tools(tools)

prompt_text = """
あなたは役に立つアシスタントです。必要に応じて最適なツールを使用してください。
ツール呼び出しを行った後に回答する場合（ツール結果にURLが含まれる場合のみ）は以下を行ってください。
・回答の末尾に参照したsource番号（例：[1][2]）を付ける
・最後にsource番号とurlを箇条書きで列挙する
"""

# プロンプトテンプレートはシンプルに構築
# SystemMessageを追加しておくと挙動が安定します
prompt = ChatPromptTemplate.from_messages([
    ("system", prompt_text),
    MessagesPlaceholder(variable_name="messages")
])
my_chain = prompt | model_with_tools

# チャットボットノード
def chatbot(state: State) -> dict[str, Any]:
    response = my_chain.invoke(state)
    return {"messages": [response]}

# ToolNodeの使用（並列ツール実行も自動対応）
tool_node = ToolNode(tools)

In [ ]:
# グラフの構築
builder = StateGraph(State)

# ノードの追加
builder.add_node("chatbot", chatbot)
builder.add_node("tools", tool_node) # 名前は慣習的に "tools" とすることが多い

# エッジの定義
builder.add_edge(START, "chatbot")

# 条件付きエッジ (Routerの自動化)
# 「ツール呼び出しがあるなら tools へ、なければ終了」というロジックは
# prebuiltの tools_condition がやってくれます。
builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)

# ツール実行後は必ずチャットボット（LLM）に戻って結果を要約させる
builder.add_edge("tools", "chatbot")

# Checkpointerの初期化
memory = InMemorySaver()

# コンパイル時にcheckpointerを渡す
search_agent = builder.compile(checkpointer=memory)

# グラフの可視化
display(Image(search_agent.get_graph().draw_mermaid_png()))

## 4.メインループ

In [ ]:
# ランダムなUUIDを生成して thread_id に設定
config = {"configurable": {"thread_id": str(uuid.uuid4())}}

while True:
    user_input = input("メッセージを入力:")
    if user_input.strip() == "":
        break
    print(f"質問：{user_input}")

    # 検索中フラグ
    is_spinning = False

    # エージェントを実行し、応答をストリーミング表示
    for chunk, metadata in search_agent.stream(
        {"messages": [{"role": "user", "content": user_input}]},
        config=config,
        stream_mode="messages"
    ):
        # 1. AIメッセージ（LLMの出力）のみを対象とする
        if isinstance(chunk, AIMessageChunk):
            # 2. ツール呼び出しの定義（JSON生成）中は表示しない
            if chunk.tool_call_chunks:
                if chunk.tool_call_chunks[-1]["name"] is not None:
                    print(f"[DEBUG]ツール呼び出し:{chunk.tool_call_chunks[-1]['name']}", flush=True)
                    is_spinning = True
                continue

            # 3. コンテンツ（最終回答のテキスト）が含まれている場合のみ表示
            if chunk.content:
                if is_spinning:
                    print() # 初回のみ改行を入れる
                    is_spinning = False
                print(chunk.content, end="", flush=True)
    print() # 改行
            
print("\n---ご利用ありがとうございました！---")